In [43]:
import pandas as pd
import numpy as np

## Import

Downloaded the  Kaggle's [dataset](https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand) and then imported as a CV

In [44]:
table = pd.read_csv("./hotel_bookings.csv")
table.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


## Data cleaning

Since the dataset have the date in different columns (day, month and year) and the month is string form, I replaced with its number and combine three of the columns in just one for creating time series in Tableau

In [45]:
# Creating a full date field for time series
meses_a_numero = {
    'January': '01', 'February': '02', 'March': '03', 'April': '04',
    'May': '05', 'June': '06', 'July': '07', 'August': '08',
    'September': '09', 'October': '10', 'November': '11', 'December': '12'
}

table["arrival_date_num_month"] = table["arrival_date_month"].map(meses_a_numero)
full_date =table["arrival_date_day_of_month"].astype(str) + "/" + table["arrival_date_num_month"] + "/" + table["arrival_date_year"].astype(str)
table.insert(3,"arrival_date",full_date)
table['reservation_status_date'] = table['reservation_status_date'].str.replace(r'(\d{4})-(\d{2})-(\d{2})', r'\3/\2/\1', regex=True)

table

,hotel,is_canceled,lead_time,arrival_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date_num_month
0,Resort Hotel,0,342,1/07/2015,2015,July,27,1,0,0,...,NaN,NaN,0,Transient,0.00,0,0,Check-Out,01/07/2015,07
1,Resort Hotel,0,737,1/07/2015,2015,July,27,1,0,0,...,NaN,NaN,0,Transient,0.00,0,0,Check-Out,01/07/2015,07
2,Resort Hotel,0,7,1/07/2015,2015,July,27,1,0,1,...,NaN,NaN,0,Transient,75.00,0,0,Check-Out,02/07/2015,07
3,Resort Hotel,0,13,1/07/2015,2015,July,27,1,0,1,...,304.0,NaN,0,Transient,75.00,0,0,Check-Out,02/07/2015,07
4,Resort Hotel,0,14,1/07/2015,2015,July,27,1,0,2,...,240.0,NaN,0,Transient,98.00,0,1,Check-Out,03/07/2015,07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,30/08/2017,2017,August,35,30,2,5,...,394.0,NaN,0,Transient,96.14,0,0,Check-Out,06/09/2017,08
119386,City Hotel,0,102,31/08/2017,2017,August,35,31,2,5,...,9.0,NaN,0,Transient,225.43,0,2,Check-Out,07/09/2017,08
119387,City Hotel,0,34,31/08/2017,2017,August,35,31,2,5,...,9.0,NaN,0,Transient,157.71,0,4,Check-Out,07/09/2017,08
119388,City Hotel,0,109,31/08/2017,2017,August,35,31,2,5,...,89.0,NaN,0,Transient,104.40,0,0,Check-Out,07/09/2017,08


Drop the useless columns and the empty cells in the country column

In [46]:
table.drop(columns=["total_of_special_requests", "required_car_parking_spaces","market_segment","babies","previous_cancellations","meal","previous_bookings_not_canceled","reservation_status", "arrival_date_week_number", "days_in_waiting_list", "arrival_date_num_month"],inplace=True)
table.dropna(axis=0,subset="country",inplace=True)
table = table[table["country"] != "TMP"]

## Data normalizing

Since the company and agent numbers are codes, the decimal places are removed from the float values, and they are converted to strings to analyze which company or agent has the most sales. Similarly, the word "Hotel" is removed from the hotel column for relevance, and the children column is converted to integers.

In [ ]:
table["company"] = pd.to_numeric(table["company"],errors="coerce").replace(np.NaN,0).astype(int).astype(str)
table["children"] = pd.to_numeric(table["children"],errors="coerce").replace(np.NaN,0).astype(int)
table["agent"] = pd.to_numeric(table["agent"],errors="coerce").replace(np.NaN,0).astype(int).astype(str)
table["hotel"] = table["hotel"].str.replace(" Hotel","")


## Internal data analysis

Verified the data in the table to know what else to change before doing the cleaning or normalizing again and then, exported it.

In [48]:
table.dtypes

hotel                         object
is_canceled                    int64
lead_time                      int64
arrival_date                  object
arrival_date_year              int64
arrival_date_month            object
arrival_date_day_of_month      int64
stays_in_weekend_nights        int64
stays_in_week_nights           int64
adults                         int64
children                       int64
country                       object
distribution_channel          object
is_repeated_guest              int64
reserved_room_type            object
assigned_room_type            object
booking_changes                int64
deposit_type                  object
agent                         object
company                       object
customer_type                 object
adr                          float64
reservation_status_date       object
dtype: object

In [49]:
table.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'country', 'distribution_channel', 'is_repeated_guest',
       'reserved_room_type', 'assigned_room_type', 'booking_changes',
       'deposit_type', 'agent', 'company', 'customer_type', 'adr',
       'reservation_status_date'],
      dtype='object')

In [50]:
table["country"].value_counts()

country
PRT    48590
GBR    12129
FRA    10415
ESP     8568
DEU     7287
       ...  
BHS        1
UMI        1
SMR        1
BWA        1
NAM        1
Name: count, Length: 176, dtype: int64

In [51]:
table[table["country"] == "USA"].loc[:,"adr"].mean()

122.9922508345255

In [52]:
table

,hotel,is_canceled,lead_time,arrival_date,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,is_repeated_guest,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,customer_type,adr,reservation_status_date
0,Resort,0,342,1/07/2015,2015,July,1,0,0,2,...,0,C,C,3,No Deposit,0,0,Transient,0.00,01/07/2015
1,Resort,0,737,1/07/2015,2015,July,1,0,0,2,...,0,C,C,4,No Deposit,0,0,Transient,0.00,01/07/2015
2,Resort,0,7,1/07/2015,2015,July,1,0,1,1,...,0,A,C,0,No Deposit,0,0,Transient,75.00,02/07/2015
3,Resort,0,13,1/07/2015,2015,July,1,0,1,1,...,0,A,A,0,No Deposit,304,0,Transient,75.00,02/07/2015
4,Resort,0,14,1/07/2015,2015,July,1,0,2,2,...,0,A,A,0,No Deposit,240,0,Transient,98.00,03/07/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City,0,23,30/08/2017,2017,August,30,2,5,2,...,0,A,A,0,No Deposit,394,0,Transient,96.14,06/09/2017
119386,City,0,102,31/08/2017,2017,August,31,2,5,3,...,0,E,E,0,No Deposit,9,0,Transient,225.43,07/09/2017
119387,City,0,34,31/08/2017,2017,August,31,2,5,2,...,0,D,D,0,No Deposit,9,0,Transient,157.71,07/09/2017
119388,City,0,109,31/08/2017,2017,August,31,2,5,2,...,0,A,A,0,No Deposit,89,0,Transient,104.40,07/09/2017


## Export

In [53]:
table.to_excel("./tableau_template.xlsx",index=False)